In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

In [2]:
BATCH_SIZE = 256
train_rate = 0.1

In [3]:
# load data without header
data = pd.read_csv('./ISWC21_data_plus_raw/rwhar_1sbj_data.csv', header=None)
# add header
data.columns = ['subject', 'x', 'y', 'z', 'activity']
data.head()

,subject,x,y,z,activity
0,0,0.378284,10.168175,0.847547,climbing_up
1,0,0.383671,10.172364,0.849942,climbing_up
2,0,0.372298,10.181941,0.859518,climbing_up
3,0,0.342969,10.170568,0.834379,climbing_up
4,0,0.319626,10.159795,0.818817,climbing_up


In [4]:
data.shape

(221621, 5)

In [5]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['activity'])
data['encoded_activity'] = encoded_labels

In [6]:
data.head()

,subject,x,y,z,activity,encoded_activity
0,0,0.378284,10.168175,0.847547,climbing_up,1
1,0,0.383671,10.172364,0.849942,climbing_up,1
2,0,0.372298,10.181941,0.859518,climbing_up,1
3,0,0.342969,10.170568,0.834379,climbing_up,1
4,0,0.319626,10.159795,0.818817,climbing_up,1


In [7]:
def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array along with the mode label for each window.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows, indices, and labels
    """
    windows = []
    indices = []
    labels = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * win_len)
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        window = data[curr:curr + win_len]
        windows.append(window.iloc[:, :-2])  # Exclude the last two columns (original and encoded labels)
        indices.append([curr, curr + win_len])
        
        # Extract and compute the mode of the encoded labels for the current window
        window_labels = window['encoded_activity']
        mode_result = mode(window_labels)
        window_label = mode_result[0] if mode_result[0].size > 0 else mode_result
        labels.append(window_label)

        curr += win_len - overlapping_elements

    result_windows = np.array(windows)
    result_indices = np.array(indices)
    result_labels = np.array(labels)
    return result_windows, result_indices, result_labels


In [8]:
sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

window_data, _, window_label = sliding_window_samples(data, window_size, overlap_ratio)
print(f"shape of window dataset (2 sec with 0% overlap): {window_data.shape}")
print(f"shape of window label (2 sec with 0% overlap): {window_label.shape}")

shape of window dataset (2 sec with 0% overlap): (2216, 100, 4)
shape of window label (2 sec with 0% overlap): (2216,)


In [9]:
#remove the subject column
window_data = window_data[:, :, 1:]

In [10]:
window_data[0].shape

(100, 3)

In [11]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
import numpy as np


window_data = window_data.astype(np.float32)

# Convert to PyTorch tensors
window_data_tensor = torch.from_numpy(window_data)
window_label_tensor = torch.from_numpy(window_label)
#convert labels to long
window_label_tensor = window_label_tensor.long()

# split data into train and test sets
train_size = int(train_rate * len(window_data_tensor))
test_size = len(window_data_tensor) - train_size

# Creating datasets
dataset = TensorDataset(window_data_tensor, window_label_tensor)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Function to extract tensors from Subset
def extract_subset_data(subset, dataset):
    return dataset.tensors[0][subset.indices], dataset.tensors[1][subset.indices]

# Extract data and labels from train and test sets
train_data, train_labels = extract_subset_data(train_dataset, dataset)
test_data, test_labels = extract_subset_data(test_dataset, dataset)

# create train and test TensorDataset
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

# create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
#  print the shape of train_loader and test_loader
print(f"shape of train_loader: {len(train_loader)}")
print(f"shape of test_loader: {len(test_loader)}")

shape of train_loader: 1
shape of test_loader: 8


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        
        res = x if self.downsample is None else self.downsample(x)

        # Adjusting the length of the residual to match the output
        if out.size(2) != res.size(2):
            desired_length = out.size(2)
            res = res[:, :, :desired_length]

        return self.relu(out + res)


class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout=0.2, num_classes=4):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size + (dilation_size - 1))]

        self.tcn = nn.Sequential(*layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        x = self.tcn(x)
        x = F.avg_pool1d(x, x.size(2)).squeeze(2)  # Global Average Pooling
        x = self.dropout(x)
        return self.fc(x)

In [15]:
# create training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.transpose(1, 2)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [16]:
# create testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    #calculate accuracy
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.transpose(1, 2)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            #calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return running_loss / len(test_loader), accuracy

In [17]:
# create function to train and test model
def train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    test_losses = []
    test_accuracies = []
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        print(f"Epoch: {epoch + 1}/{num_epochs}.. Train Loss: {train_loss:.3f}.. Test Loss: {test_loss:.3f}.. Test Accuracy: {test_accuracy:.3f}")
    return train_losses, test_losses, test_accuracies

In [18]:
# get number of classes
num_classes = len(np.unique(window_label))
print(f"number of classes: {num_classes}")

number of classes: 8


In [19]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/50.. Train Loss: 2.092.. Test Loss: 2.186.. Test Accuracy: 0.138
Epoch: 2/50.. Train Loss: 2.161.. Test Loss: 1.873.. Test Accuracy: 0.283
Epoch: 3/50.. Train Loss: 1.867.. Test Loss: 1.823.. Test Accuracy: 0.618
Epoch: 4/50.. Train Loss: 1.817.. Test Loss: 1.721.. Test Accuracy: 0.567
Epoch: 5/50.. Train Loss: 1.701.. Test Loss: 1.624.. Test Accuracy: 0.476
Epoch: 6/50.. Train Loss: 1.594.. Test Loss: 1.482.. Test Accuracy: 0.566
Epoch: 7/50.. Train Loss: 1.453.. Test Loss: 1.328.. Test Accuracy: 0.620
Epoch: 8/50.. Train Loss: 1.302.. Test Loss: 1.200.. Test Accuracy: 0.704
Epoch: 9/50.. Train Loss: 1.169.. Test Loss: 1.065.. Test Accuracy: 0.749
Epoch: 10/50.. Train Loss: 1.016.. Test Loss: 0.956.. Test Accuracy: 0.796
Epoch: 11/50.. Train Loss: 0.886.. Test Loss: 0.868.. Test Accuracy: 0.750
Epoch: 12/50.. Train Loss: 0.778.. Test Loss: 0.795.. Test Accuracy: 0.787
Epoch: 13/50.. Train Loss: 0.677.. Test Loss: 0.763.. Test Accuracy: 0.786
Epoch: 14/50.. Train Loss: 0.621..

In [20]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)


Epoch: 1/50.. Train Loss: 2.189.. Test Loss: 2.230.. Test Accuracy: 0.143
Epoch: 2/50.. Train Loss: 2.167.. Test Loss: 2.208.. Test Accuracy: 0.144
Epoch: 3/50.. Train Loss: 2.144.. Test Loss: 2.186.. Test Accuracy: 0.144
Epoch: 4/50.. Train Loss: 2.122.. Test Loss: 2.165.. Test Accuracy: 0.144
Epoch: 5/50.. Train Loss: 2.101.. Test Loss: 2.144.. Test Accuracy: 0.146
Epoch: 6/50.. Train Loss: 2.080.. Test Loss: 2.123.. Test Accuracy: 0.148
Epoch: 7/50.. Train Loss: 2.060.. Test Loss: 2.104.. Test Accuracy: 0.153
Epoch: 8/50.. Train Loss: 2.039.. Test Loss: 2.084.. Test Accuracy: 0.162
Epoch: 9/50.. Train Loss: 2.020.. Test Loss: 2.066.. Test Accuracy: 0.171
Epoch: 10/50.. Train Loss: 2.001.. Test Loss: 2.047.. Test Accuracy: 0.185
Epoch: 11/50.. Train Loss: 1.982.. Test Loss: 2.030.. Test Accuracy: 0.198
Epoch: 12/50.. Train Loss: 1.964.. Test Loss: 2.012.. Test Accuracy: 0.212
Epoch: 13/50.. Train Loss: 1.946.. Test Loss: 1.996.. Test Accuracy: 0.229
Epoch: 14/50.. Train Loss: 1.929..

In [21]:
num_inputs = 3  # Assuming 3 input channels (x, y, z axes of the accelerometer)
num_channels = [64, 128, 256]  # Example channel sizes for each layer
kernel_size = 8  # Kernel size for temporal convolutions

model = TCN(num_inputs, num_channels, kernel_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/50.. Train Loss: 2.057.. Test Loss: 1.796.. Test Accuracy: 0.287
Epoch: 2/50.. Train Loss: 1.800.. Test Loss: 1.606.. Test Accuracy: 0.393
Epoch: 3/50.. Train Loss: 1.565.. Test Loss: 1.460.. Test Accuracy: 0.394
Epoch: 4/50.. Train Loss: 1.515.. Test Loss: 1.205.. Test Accuracy: 0.638
Epoch: 5/50.. Train Loss: 1.170.. Test Loss: 1.181.. Test Accuracy: 0.483
Epoch: 6/50.. Train Loss: 1.105.. Test Loss: 0.989.. Test Accuracy: 0.647
Epoch: 7/50.. Train Loss: 0.940.. Test Loss: 0.987.. Test Accuracy: 0.651
Epoch: 8/50.. Train Loss: 0.922.. Test Loss: 0.916.. Test Accuracy: 0.598
Epoch: 9/50.. Train Loss: 0.809.. Test Loss: 0.885.. Test Accuracy: 0.675
Epoch: 10/50.. Train Loss: 0.770.. Test Loss: 0.812.. Test Accuracy: 0.820
Epoch: 11/50.. Train Loss: 0.714.. Test Loss: 0.792.. Test Accuracy: 0.748
Epoch: 12/50.. Train Loss: 0.639.. Test Loss: 0.873.. Test Accuracy: 0.684
Epoch: 13/50.. Train Loss: 0.599.. Test Loss: 0.807.. Test Accuracy: 0.749
Epoch: 14/50.. Train Loss: 0.669..

In [22]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_20231216-0131.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)

Epoch: 1/50.. Train Loss: 2.149.. Test Loss: 2.091.. Test Accuracy: 0.069
Epoch: 2/50.. Train Loss: 2.108.. Test Loss: 2.065.. Test Accuracy: 0.043
Epoch: 3/50.. Train Loss: 2.089.. Test Loss: 2.043.. Test Accuracy: 0.071
Epoch: 4/50.. Train Loss: 2.079.. Test Loss: 2.024.. Test Accuracy: 0.147
Epoch: 5/50.. Train Loss: 2.036.. Test Loss: 2.005.. Test Accuracy: 0.159
Epoch: 6/50.. Train Loss: 2.017.. Test Loss: 1.988.. Test Accuracy: 0.174
Epoch: 7/50.. Train Loss: 1.996.. Test Loss: 1.972.. Test Accuracy: 0.196
Epoch: 8/50.. Train Loss: 1.970.. Test Loss: 1.956.. Test Accuracy: 0.205
Epoch: 9/50.. Train Loss: 1.959.. Test Loss: 1.941.. Test Accuracy: 0.217
Epoch: 10/50.. Train Loss: 1.946.. Test Loss: 1.926.. Test Accuracy: 0.235
Epoch: 11/50.. Train Loss: 1.929.. Test Loss: 1.912.. Test Accuracy: 0.240
Epoch: 12/50.. Train Loss: 1.910.. Test Loss: 1.898.. Test Accuracy: 0.238
Epoch: 13/50.. Train Loss: 1.901.. Test Loss: 1.884.. Test Accuracy: 0.231
Epoch: 14/50.. Train Loss: 1.871..